# A functional burst search and download service

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

import time
from datetime import datetime

import asf_search as asf
import geopandas as gpd
import numpy as np
import pystac
import xarray as xr
from shapely import wkt

import bursts

In [ ]:
safe_url = 'https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SDV_20220716T115019_20220716T115049_044125_054459_71A9.zip'

## Demoing interior objects
These objects will not likely not be accessed by users, but this function demonstrates how the metadata objects are created

In [ ]:
auth = bursts.get_netrc_auth()
manifest, annotations = bursts.edl_download_metadata(safe_url, auth)

In [ ]:
slc = bursts.SLCMetadata(safe_url, manifest, annotations)
swath = bursts.SwathMetadata(slc,'vv',swath_index=0)
burst = bursts.BurstMetadata(swath,burst_index=0)

In [ ]:
item = burst.to_stac_item()
item.validate()

In [ ]:
item.stac_extensions

## Creating burst metadata datasets
Users have the option of creating either a geodatframe, or a STAC catalog. Burst data download is only enabled for the STAC catalog, but the geodataframe is great for in-notebook visualizations. Hopefully, users won't have to create these datasets themselves and will instead use an ASF-provided dataset.

In [ ]:
start = datetime.strptime('20210101','%Y%m%d')
end = datetime.strptime('20210601','%Y%m%d')
mt_edgecumbe = 'POLYGON ((-135.7849 57.0417,-135.7243 57.0417,-135.7243 57.0654,-135.7849 57.0654,-135.7849 57.0417))'
insar_opts = dict(platform=[asf.PLATFORM.SENTINEL1], polarization=['VV','VV+VH','Dual VV'], beamMode='IW', processingLevel='SLC')
results = asf.geo_search(flightDirection='DESCENDING', intersectsWith=mt_edgecumbe, start=start, end=end, **insar_opts)
safe_urls = [x.properties['url'] for x in results]
len(safe_urls)

In [ ]:
safe_urls

In [ ]:
burst_list = bursts.get_burst_metadata(safe_urls, threads=10)

In [ ]:
stac = bursts.generate_burst_stac_catalog(burst_list)
stac.describe()

## Explore with Geopandas

In [ ]:
items = [x.to_dict() for x in stac.get_all_items()]
df = gpd.GeoDataFrame.from_features(items, crs="epsg:4326")
df.index = [x['id'] for x in items]

In [ ]:
in_aoi = df.intersects(wkt.loads(mt_edgecumbe))
aoi = df.loc[in_aoi].copy()
print(f'Located {aoi.shape[0]} bursts that meet criteria')

In [ ]:
aoi.explore(style_kwds={'fill':None})

## View using STAC-browser

In [ ]:
stac_path = bursts.save_stac_catalog_locally(stac)

In [ ]:
bursts.initiate_stac_catalog_server(5555, stac_path.parent)

## Download burst data
Using the STAC catalog, users can select an STAC item, then pass this item to `bursts.edl_download_burst_data`. The data is returned in a 2d numpy array with a complex float data type.

In [ ]:
item_list = [stac.get_item(x,recursive=True) for x in aoi.index]

In [ ]:
stack_dataset = bursts.edl_download_stack(item_list, threads=10)

In [ ]:
stack_dataset

In [ ]:
stack_dataset.to_zarr('example.zarr')

In [ ]:
read_back_in = xr.open_zarr('example.zarr')

In [ ]:
read_back_in

Now you have a zarr/xarrray dataset representing a Sentinel-1 burst (STAC)k!